In [6]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import Preprocess

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Masking

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [17]:
data = pd.read_csv("Data/Electrical_Averaged.csv")
data["Timestamp"] = pd.to_datetime(data['Timestamp'])
data.set_index('Timestamp', inplace=True)
building_data = Preprocess.remove_outliers(data, 15)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(building_data)

n_steps_in = 30
n_steps_out = 1
mask_value = 0
tf.random.set_seed(1)

# set check point
checkpoint_path = f"training_1/checkpoint__{n_steps_in}in_{n_steps_out}_out"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

def generate_sequence(data, n_steps_in, n_steps_out):
    X, y = list(), list()
    for index in range(n_steps_in, len(data)-n_steps_out+1):
        X.append(data[index-n_steps_in : index])
        y.append(data[index : index+n_steps_out])
    return np.array(X), np.array(y)

scaled_data[np.isnan(scaled_data)] = mask_value
X, y = generate_sequence(scaled_data, n_steps_in, n_steps_out)

n_features = X.shape[2]
print(scaled_data.shape, X.shape, y.shape)

# define model
model = Sequential()
model.add(Masking(mask_value= mask_value, input_shape=(n_steps_in, n_features)))
model.add(LSTM(200, activation='relu'))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

(3661, 12) (3631, 30, 12) (3631, 1, 12)


In [18]:
# fit model
model.fit(X, y, epochs=200, verbose=0, callbacks=[cp_callback])
# Save the weights
model.save_weights(f'./Weight/weight_{n_steps_in}in_{n_steps_out}_out')


Epoch 1: saving model to training_1/checkpoint__30in_1_out

Epoch 2: saving model to training_1/checkpoint__30in_1_out

Epoch 3: saving model to training_1/checkpoint__30in_1_out

Epoch 4: saving model to training_1/checkpoint__30in_1_out

Epoch 5: saving model to training_1/checkpoint__30in_1_out

Epoch 6: saving model to training_1/checkpoint__30in_1_out

Epoch 7: saving model to training_1/checkpoint__30in_1_out

Epoch 8: saving model to training_1/checkpoint__30in_1_out

Epoch 9: saving model to training_1/checkpoint__30in_1_out

Epoch 10: saving model to training_1/checkpoint__30in_1_out

Epoch 11: saving model to training_1/checkpoint__30in_1_out

Epoch 12: saving model to training_1/checkpoint__30in_1_out

Epoch 13: saving model to training_1/checkpoint__30in_1_out

Epoch 14: saving model to training_1/checkpoint__30in_1_out

Epoch 15: saving model to training_1/checkpoint__30in_1_out

Epoch 16: saving model to training_1/checkpoint__30in_1_out

Epoch 17: saving model to traini

In [19]:
# Restore the weights
# model.load_weights(f'./Weight/weight_{n_steps_in}in_{n_steps_out}_out')

ahead = 365

def predict_ahead(X, model, n_ahead):
    yhat = list()
    X_copy = np.copy(X)
    for _ in range(n_ahead):
        prediction = model.predict(X_copy, verbose=0)
        yhat.append(prediction[0, 0])
        X_copy = np.concatenate((X_copy, prediction), axis=1)[:,1:]
    return np.array(yhat)
yhat = predict_ahead(X[-1:], model, ahead)    
yhat = scaler.inverse_transform(yhat)
time_period = pd.date_range(building_data.index[-1], periods=ahead+1, freq="d")[1:]
yhat = pd.DataFrame(yhat, columns = building_data.columns, index = time_period)
yhat.index.name = "Timestamp"

In [20]:
yhat.index.name = "Timestamp"
yhat.to_csv("Data/Electrical_Predict.csv")
yhat.reset_index(drop=True, inplace=True)
yhat = pd.DataFrame(yhat, columns = building_data.columns)
# df.append(data_to_append, ignore_index = True)

In [ ]:
Reference
# model: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
# model: https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
# predict ahead: https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816